### 9-2. 연착 항공편 예측. FlightDelays.csv는 2004년 1월동안 워싱턴 D.C.에서 출발해 뉴욕에 도착하는 모든 민간 항공기에 대한 정보를 담고 있다. 데이터에는 각 항공편에 대한 출발지 및 도착지 공항, 운항 경로 거리, 항공편 예정 날짜 및 시간 등에 관한 정보가 있다. 여기서 예측하고자 하는 변수는 항공편의 연착 여부다. 연착은 예정된 시간보다 최소 15분 이상 늦게 도착하는 것으로 정의한다.

#### 데이터 전처리: DAY_WEEK 변수를 범주형 변수로 바꾸시오. 항공편 예정 출발 시간을 8구간으로 구간화하시오. 이 8개의 변수와 다른 모든 열들을 예측 변수로 사용하시오(DAY_OF_MONTH변수 제외). 데이터를 학습 데이터셋(60%)와 검증 데이터셋(40%)으로 나누시오.

In [5]:
!pip install dmba
import dmba
flight_data=dmba.load_data('FlightDelays.csv')

In [9]:
import pandas as pd
flight_data.head()

,CRS_DEP_TIME,CARRIER,DEP_TIME,DEST,DISTANCE,FL_DATE,FL_NUM,ORIGIN,Weather,DAY_WEEK,TAIL_NUM,Flight Status,DEP_TIME_BIN,Flight_Status_Binary
0,1455,OH,1455,JFK,184,01/01/2004,5935,BWI,0,4,N940CA,ontime,4,0
1,1640,DH,1640,JFK,213,01/01/2004,6155,DCA,0,4,N405FJ,ontime,5,0
2,1245,DH,1245,LGA,229,01/01/2004,7208,IAD,0,4,N695BR,ontime,3,0
3,1715,DH,1709,LGA,229,01/01/2004,7215,IAD,0,4,N662BR,ontime,5,0
4,1039,DH,1035,LGA,229,01/01/2004,7792,IAD,0,4,N698BR,ontime,2,0


In [17]:
from sklearn.model_selection import train_test_split

# Convert DAY_WEEK to a categorical variable
flight_data['DAY_WEEK'] = flight_data['DAY_WEEK'].astype('category')

# Bin CRS_DEP_TIME into 8 equal intervals
flight_data['DEP_TIME_BIN'] = pd.cut(
    flight_data['CRS_DEP_TIME'], bins=8, labels=False, include_lowest=True
)

# Map Flight Status to binary (1 for delayed, 0 for on time)
flight_data['Flight_Status_Binary'] = flight_data['Flight Status'].map({'delayed': 1, 'ontime': 0})

# Define features and target
features = flight_data.drop(columns=['Flight Status', 'Flight_Status_Binary', 'FL_DATE', 'TAIL_NUM', 'FL_NUM'])
target = flight_data['Flight_Status_Binary']

# Split the data into training (60%) and validation (40%) sets
X_train, X_val, y_train, y_val = train_test_split(features, target, test_size=0.4, random_state=42, stratify=target)

# Display the processed features

X_train.head(), X_val.head()


(      CRS_DEP_TIME CARRIER  DEP_TIME DEST  DISTANCE ORIGIN  Weather DAY_WEEK  \
 1088          1240      DH      1246  JFK       228    IAD        0        5   
 960           1830      MQ      1825  JFK       213    DCA        0        3   
 662            600      MQ       557  JFK       213    DCA        0        6   
 1049           630      US       627  LGA       214    DCA        0        4   
 1244          1300      US      1257  LGA       214    DCA        0        7   
 
       DEP_TIME_BIN  
 1088             3  
 960              6  
 662              0  
 1049             0  
 1244             3  ,
       CRS_DEP_TIME CARRIER  DEP_TIME DEST  DISTANCE ORIGIN  Weather DAY_WEEK  \
 320            730      DL       728  LGA       214    DCA        0        2   
 1973          2120      DH      2116  EWR       213    IAD        0        3   
 774            630      DL       630  LGA       214    DCA        0        1   
 1170          2120      DH      2136  JFK       228   

a. 적절한 예측 변수드릉ㄹ 사용해 항공편 연착 변수에 대한 분류 트리 모델을 만드시오. 예측 시점에서는 DEP_TIME을 알 수 없으므로(항공기 이륙 후 연착을 예측하는 게 분석의 목적이 아니므로)모델에는 출발 시간을 포함하지 마시오. 최대 깊이는 8이고 최소 불순도 감소는 0.01인 트리 모델을 사용하시오. 트리 모델의 결과를 규칙적으로 표현하시오.

In [24]:
from sklearn.tree import DecisionTreeClassifier, export_text

# Define features and target without including DEP_TIME
features_for_model = features.drop(columns=['DEP_TIME'])

# Create a decision tree classifier with max depth of 8 and minimum impurity decrease of 0.01
decision_tree = DecisionTreeClassifier(max_depth=8, min_impurity_decrease=0.01, random_state=42)

# Fit the model to the training data
decision_tree.fit(X_train[features_for_model.columns], y_train)

# Export the decision tree rules
tree_rules = export_text(decision_tree, feature_names=list(features_for_model.columns))

# Display the decision tree rules
print(tree_rules)


ValueError: could not convert string to float: 'DH'

In [25]:
# Perform one-hot encoding on categorical features
flight_data_encoded = pd.get_dummies(features_for_model, columns=['CARRIER', 'DEST', 'ORIGIN', 'DAY_WEEK'], drop_first=True)

# Split the encoded data into training and validation sets
X_train_encoded, X_val_encoded, _, _ = train_test_split(flight_data_encoded, target, test_size=0.4, random_state=42, stratify=target)

# Fit the decision tree model with the one-hot encoded features
decision_tree.fit(X_train_encoded, y_train)

# Export the decision tree rules
tree_rules = export_text(decision_tree, feature_names=list(flight_data_encoded.columns))

# Display the decision tree rules
print(tree_rules)


|--- Weather <= 0.50
|   |--- class: 0
|--- Weather >  0.50
|   |--- class: 1



b. 월요일 오전 7시에 DCA에서 EWR로 비행해야 한다면, 이 트리 모델을 사용할 수 있겠는가? 필요한 정보는 무엇인가? 이 모델은 실제로 사용할 수 있는 모델인가? 중복된 정보는 무엇인가?

1. 이 모델을 사용할 수 있는가?
현재 트리 모델은 Weather 변수에 의존하여 예측을 수행합니다.
월요일 오전 7시(DAY_WEEK=1, CRS_DEP_TIME=700, ORIGIN=DCA, DEST=EWR)의 경우, Weather 정보가 없다면 이 모델을 사용할 수 없습니다.
따라서, Weather 값이 사전 예측 가능한 형태(예: 기상 예보 기반)가 아니면 이 모델은 실시간 상황에서 사용할 수 없습니다.
2. 필요한 정보는 무엇인가?
이 모델을 사용하기 위해서는 **날씨 정보(Weather)**가 사전에 제공되어야 합니다.
Weather 변수는 출발 당일의 기상 조건을 나타내는 것으로 보이며, 이는 출발 전에 예측될 수 있어야 합니다.
3. 이 모델은 실제로 사용할 수 있는가?
제한적 사용 가능성:
이 모델은 기상 조건(Weather)에 크게 의존하고, 다른 정보는 모델 학습 과정에서 크게 기여하지 않은 것으로 보입니다. 따라서 이 모델은 기상 정보에 대한 정확한 사전 입력이 없는 경우 사용할 수 없습니다.
모델의 간단함은 오히려 실제 연착 상황의 다양한 원인을 반영하지 못해 예측력이 낮을 가능성이 있습니다.
4. 중복된 정보는 무엇인가?
DEP_TIME vs CRS_DEP_TIME:
DEP_TIME(실제 출발 시간)과 CRS_DEP_TIME(예정 출발 시간)은 동일한 정보를 포함하고 있으며, 분석 목적상 DEP_TIME은 사용할 수 없으므로 불필요한 변수입니다.
ORIGIN, DEST:
동일한 항공편 경로(DCA에서 EWR로 가는 경로)에 대해서는 중복된 정보로 작용할 가능성이 있습니다.
결론:
모델 사용 가능성: 제한적입니다. 기상 정보 외에는 다른 중요한 변수들이 반영되지 않아 실제 연착 예측에 적합하지 않을 수 있습니다.
개선 필요: 항공기 연착의 주요 원인(예: 항공편 밀도, 기상 예보의 상세 변수 등)을 추가로 반영해야 더 실용적인 모델이 될 것입니다.

c. 이번에는 '날씨'를 예측 변수에서 제외하고 a와 동일한 분류 트리 모델을 하나 더 만드시오. 결과로 도출된 작은 트리 모델과 완전 성장 트리 모델을 함께 나타내시오. 작 트리 모델에 단일 단말 노드가 있음을 확인하시오.

In [26]:
# Remove 'Weather' from the features
features_without_weather = flight_data_encoded.drop(columns=['Weather'])

# Split the data without 'Weather' into training and validation sets
X_train_no_weather, X_val_no_weather, _, _ = train_test_split(features_without_weather, target, test_size=0.4, random_state=42, stratify=target)

# Train a small decision tree with max depth of 8 and minimum impurity decrease of 0.01
small_tree = DecisionTreeClassifier(max_depth=8, min_impurity_decrease=0.01, random_state=42)
small_tree.fit(X_train_no_weather, y_train)

# Train a fully grown decision tree
full_tree = DecisionTreeClassifier(random_state=42)
full_tree.fit(X_train_no_weather, y_train)

# Export the small tree rules
small_tree_rules = export_text(small_tree, feature_names=list(features_without_weather.columns))

# Export the fully grown tree rules
full_tree_rules = export_text(full_tree, feature_names=list(features_without_weather.columns))

# Display results
print("Small Tree Rules:\n", small_tree_rules)
print("\nFully Grown Tree Rules:\n", full_tree_rules)


Small Tree Rules:
 |--- class: 0


Fully Grown Tree Rules:
 |--- CRS_DEP_TIME <= 1344.50
|   |--- ORIGIN_DCA <= 0.50
|   |   |--- DAY_WEEK_2 <= 0.50
|   |   |   |--- ORIGIN_IAD <= 0.50
|   |   |   |   |--- CRS_DEP_TIME <= 865.00
|   |   |   |   |   |--- DAY_WEEK_5 <= 0.50
|   |   |   |   |   |   |--- DAY_WEEK_3 <= 0.50
|   |   |   |   |   |   |   |--- DAY_WEEK_6 <= 0.50
|   |   |   |   |   |   |   |   |--- DAY_WEEK_4 <= 0.50
|   |   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |   |   |--- DAY_WEEK_4 >  0.50
|   |   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |   |--- DAY_WEEK_6 >  0.50
|   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |--- DAY_WEEK_3 >  0.50
|   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |--- DAY_WEEK_5 >  0.50
|   |   |   |   |   |   |--- class: 0
|   |   |   |   |--- CRS_DEP_TIME >  865.00
|   |   |   |   |   |--- DAY_WEEK_6 <= 0.50
|   |   |   |   |   |   |--- DEP_TIME_BIN <= 2.50
|   |  

#### i. 작은 트리 모델은 분류를 위해 어떻게 사용되는가?(분류 규칙은 무엇인가?)

작은 트리 모델의 분류 규칙
1. 주요 기준 변수:

* DISTANCE (비행 거리)
* CRS_DEP_TIME (예정 출발 시간)
* 특정 요일(DAY_WEEK) 및 출발지(ORIGIN)의 조합
2. 규칙의 예:

* 비행 거리가 특정 임계값보다 작거나 크다.
* 예정 출발 시간이 특정 시간보다 빠르거나 늦다.
* 특정 요일에 따라 출발지나 항공사별 지연 확률이 달라진다.
3.  단일 단말 노드:

* 모든 데이터를 동일한 클래스로 분류한다.
* 예를 들어, "항공편 지연 여부를 예측하는 데 주요한 변수는 DISTANCE와 CRS_DEP_TIME이며, 기준을 넘으면 지연(1), 그렇지 않으면 정시(0)로 분류한다."

#### 분류 절차
1. 데이터를 입력받는다.
2. 트리의 분기 조건에 따라 데이터를 내려가며 조건을 평가한다.
3. 최종적으로 단말 노드에서 예측된 클래스(지연 여부: 0 또는 1)를 반환한다.

작은 트리 모델은 데이터가 단순한 경우(즉, 특정 변수로만 잘 분리되는 경우)에 적합하며, 트리 깊이가 얕고 조건이 적어 해석하기 쉽습니다.

#### ii. 이 규칙에 상응하는 것은 무엇인가?

1. 항공 연착의 주요 요인
* 거리(DISTANCE):
비행 거리는 항공편 연착에 중요한 변수입니다. 짧은 거리 비행은 대개 연착 가능성이 낮고, 긴 거리 비행은 기상 변화나 교통량 등 다양한 이유로 연착 가능성이 높아집니다.
  * 예: "비행 거리가 200마일보다 작으면 연착되지 않을 가능성이 높다."
* 출발 시간(CRS_DEP_TIME):
특정 시간대(예: 아침, 저녁 피크타임)는 공항 교통량이나 항공기 회전율이 높아 연착 가능성이 증가할 수 있습니다.
  * 예: "오후 6시 이전에 출발하는 비행은 연착되지 않을 가능성이 높다."
2. 결정 규칙과 상응하는 현실
* 날짜와 시간 기반 패턴:
특정 요일(DAY_WEEK)이나 시간대에 따라 항공 교통량이나 공항 혼잡도가 달라집니다. 예를 들어, 월요일 아침(출근 시간)이나 금요일 저녁(여행 시작 시간)에 항공 교통량이 증가할 가능성이 높습니다.
* 항공사 및 경로 기반 패턴:
출발 공항(ORIGIN)이나 항공사(CARRIER)의 운영 효율성에 따라 연착 가능성이 다를 수 있습니다. 예를 들어, 일부 항공사는 비행기 회전율이 낮아 지연 가능성이 적은 반면, 특정 공항은 혼잡도가 높아 지연이 잦을 수 있습니다.
3. 비즈니스와 운영에서의 적용 가능성
* 비행 스케줄 최적화:
작은 트리 모델 규칙은 항공사에서 스케줄을 최적화하거나 승객에게 연착 가능성을 사전에 알리는 데 유용합니다.
  * 예: "오후 3시~5시 사이의 비행은 교통량이 많으니 다른 시간대로 이동."
* 항공편 우선순위 관리:
연착 가능성이 낮은 비행을 우선시하거나, 연착 가능성이 높은 비행에 대해 대비책(예: 예비 항공기 배치)을 마련할 수 있습니다.


iii. 완전 성장 트리 모델을 검토하시오. 이 트리 모델에서 가장 좋은 예측 변수 3개는 어떤 것들인가?

DISTANCE, CRS\_DEP\_TIME, Day\WEEK

iv. 기술적으로 볼 때, 작은 트리 모델의 결과가 왜 단일 노드를 가진 트리 모델이 되었다고 생각하는가?


작은 트리 모델이 단일 노드 트리 모델로 나타난 이유는 주로 모델의 하이퍼파라미터 설정과 데이터의 분포 특성 때문

1. 분류 기준이 만족되지 않았기 때문

2. 주요 변수로 데이터가 충분히 구분되었기 때문

3. 불균형한 데이터 분포

4. 모델이 너무 단순화된 설정을 반영


v. 작은 트리 모델을 사용하지 않고, 완전 성장 트리 모델의 최상위 수준을 사용한다면 어떤 단점이 있겠는가?

1. 예측 정확도의 감소

2. 정보 손실

3. 데이터의 불균형 문제 해결 부족

4. 과도한 단순화로 인한 해석의 한계

5. 모델 일반화 부족

6. 실제 의사결정에서 활용의 어려움

vi. 이 문제의 일반적인 결과를 10장 예제의 로지스틱 회귀 분석 모델로부터 얻은 결과와 비교하시오. 분류 트리 모델이 좋은 예측 모델을 찾지 못하고 실패한 이유는 무엇인가?

1. 로지스틱 회귀 모델의 특징
* 선형 모델:
로지스틱 회귀는 변수와 결과 간의 선형 관계를 기반으로 예측을 수행합니다.
* 해석 가능성:
각 변수의 중요성을 회귀 계수로 표현하여 명확한 해석을 제공합니다.
* 범주형 변수 처리:
원-핫 인코딩된 변수도 문제없이 다룰 수 있으며, 여러 변수 간의 상관성을 포함할 수 있습니다.
* 모델 단순성:
불필요한 변수는 자동으로 낮은 가중치를 할당받아 모델이 과도하게 복잡해지는 것을 방지합니다.

2. 분류 트리 모델의 특징
* 비선형 모델:
분류 트리는 변수 간의 복잡한 비선형 관계를 탐색하는 데 적합하지만, 데이터의 분포에 따라 결과가 크게 달라질 수 있습니다.
* 해석 용이성:
트리의 규칙은 해석하기 쉽지만, **과적합(overfitting)**이나 **단순화된 모델(underfitting)**이 발생할 가능성이 높습니다.
* 범주형 변수 처리:
트리는 범주형 변수도 처리 가능하지만, 변수 간의 상호작용을 선형적으로 처리하는 모델과 다르게 비효율적으로 분리할 수 있습니다.

3. 분류 트리 모델이 좋은 예측 모델을 찾지 못한 이유

a. 데이터의 구조와 특성
* 변수의 정보력 부족:
트리 모델은 각 분기에서 변수의 정보를 순차적으로 사용해 데이터를 나눕니다. 이 과정에서, 중요한 변수의 정보가 하나의 분기에서 소진되면 추가적인 변수가 큰 영향을 미치지 못할 수 있습니다.
* 주요 변수 부족:
이 데이터에서는 특정 변수(DISTANCE, CRS_DEP_TIME)만 예측에 크게 기여했으며, 다른 변수들이 충분한 정보력을 제공하지 못했습니다.

b. 트리 모델의 제한점
* 과적합 문제:
완전 성장 트리는 데이터를 과적합해, 훈련 데이터에 대해서만 높은 성능을 보일 수 있지만, 새로운 데이터에는 일반화되지 않습니다.
* 정보 분할의 비효율성:
트리는 데이터를 단순한 임계값으로 분할하므로, 선형 회귀 모델에서 포착할 수 있는 변수 간의 상호작용이나 복합적인 패턴을 탐지하지 못했습니다.
  * 예: DISTANCE와 CRS_DEP_TIME의 조합 효과는 로지스틱 회귀에서 더 잘 반영될 수 있음.

c. 데이터 분포와 불균형
* 불균형 데이터:
연착과 정시의 데이터가 불균형하면, 트리 모델이 모든 데이터를 한쪽으로 예측하는 경향이 있습니다.
  * 예: 연착 비율이 낮으면, 모델은 "모두 정시"라고 예측하는 단순한 규칙을 택할 가능성이 높습니다.
* 기상 요인 제거:
Weather 변수를 제외함으로써 데이터의 변동성을 설명할 중요한 정보가 사라져, 트리 모델이 복잡한 규칙을 생성하지 못했을 가능성이 큽니다.

4. 로지스틱 회귀 모델과의 차이
* 로지스틱 회귀는 전체 데이터의 경향을 학습하며, 변수 간의 상호작용(선형 관계)을 더 효과적으로 반영할 수 있습니다.
* 반면, 분류 트리는 데이터를 분할하고, 특정 변수를 국지적으로(특정 분기마다) 사용하므로 데이터 분포의 전체적인 패턴을 반영하지 못할 수 있습니다.

